# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating the kernel arguments with your requests.

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The chat history is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the chat history on disk so that you can bring it into your applications.

In this chat scenario, as the user talks back and forth with the bot, the chat context gets populated with the history of the conversation. During each new run of the kernel, the kernel arguments and chat history can provide the AI with its variables' content.


In [1]:
# Note: if using a virtual environment, do not run this cell
#%pip install -U semantic-kernel
from semantic_kernel import __version__

__version__

'1.28.1'

Initial configuration for the notebook to run properly.

In [2]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

### Configuring the Kernel

Let's get started with the necessary configuration to run Semantic Kernel. For Notebooks, we require a `.env` file with the proper settings for the model you use. Create a new file named `.env` and place it in this directory. Copy the contents of the `.env.example` file from this directory and paste it into the `.env` file that you just created.

**NOTE: Please make sure to include `GLOBAL_LLM_SERVICE` set to either OpenAI, AzureOpenAI, or HuggingFace in your .env file. If this setting is not included, the Service will default to AzureOpenAI.**

#### Option 1: using OpenAI

Add your [OpenAI Key](https://openai.com/product/) key to your `.env` file (org Id only if you have multiple orgs):

```
GLOBAL_LLM_SERVICE="OpenAI"
OPENAI_API_KEY="sk-..."
OPENAI_ORG_ID=""
OPENAI_CHAT_MODEL_ID=""
OPENAI_TEXT_MODEL_ID=""
OPENAI_EMBEDDING_MODEL_ID=""
```
The names should match the names used in the `.env` file, as shown above.

#### Option 2: using Azure OpenAI

Add your [Azure Open AI Service key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio) settings to the `.env` file in the same folder:

```
GLOBAL_LLM_SERVICE="AzureOpenAI"
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_TEXT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME="..."
AZURE_OPENAI_API_VERSION="..."
```
The names should match the names used in the `.env` file, as shown above.

For more advanced configuration, please follow the steps outlined in the [setup guide](./CONFIGURING_THE_KERNEL.md).

We will load our settings and get the LLM service to use for the notebook.

In [3]:
from services import Service

from samples.service_settings import ServiceSettings

service_settings = ServiceSettings()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


In [4]:
from semantic_kernel import Kernel

kernel = Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

Let's define a prompt outlining a dialogue chat bot.


In [5]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function


In [6]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings, OpenAIChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-35-turbo",
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.add_function(
    function_name="chat",
    plugin_name="chatPlugin",
    prompt_template_config=prompt_template_config,
)

In [7]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good about giving book recommendations.")

Initialize the Kernel Arguments


In [8]:
from semantic_kernel.functions import KernelArguments

arguments = KernelArguments(user_input="Hi, I'm looking for book suggestions", history=chat_history)

Chat with the Bot


In [9]:
response = await kernel.invoke(chat_function, arguments)
print(response)

Of course! I'd love to help. Could you tell me a bit more about what you're looking for? Are you into fiction or nonfiction? Any specific genres, themes, or authors you enjoy?


Update the history with the output


In [10]:
chat_history.add_assistant_message(str(response))

Keep Chatting!


In [11]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(answer))

In [12]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Absolutely! Here are some excellent book recommendations that delve into Greek history and philosophy:

### Greek History:
1. **"The Histories" by Herodotus**  
   Known as the "Father of History," Herodotus provides fascinating insights into ancient Greece and its neighboring civilizations, blending historical accounts with cultural observations.

2. **"The Peloponnesian War" by Thucydides**  
   A detailed and analytical account of the conflict between Athens and Sparta, Thucydides' work is considered one of the greatest historical texts ever written.

3. **"Alexander the Great" by Philip Freeman**  
   This biography offers a compelling narrative of Alexander the Great's life, conquests, and legacy, making it accessible and engaging for readers.

4. **"The Greeks: A Global History" by Roderick Beaton**  
   This recent book is a sweeping overview of Greek history, from ancient

In [13]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: Sure! Let me give you a bit more detail about **"The Greeks and the Irrational" by E.R. Dodds**:

This fascinating book challenges the traditional view of ancient Greece as a purely rational and logical society. Dodds argues that, alongside their celebrated achievements in philosophy, science, and logic, the Greeks were profoundly influenced by irrational forces such as emotions, dreams, religious experiences, and superstitions. 

Using insights from psychology and anthropology, Dodds explores themes like the role of shame and guilt in Greek society, the significance of divine possession and oracles, and how irrational beliefs shaped Greek drama, literature, and philosophy. He also examines the contrast between rational and non-rational thought in key texts, such as those by Homer, Euripides, and Plato.

This book is considered groundbreaking because it provides a more nuanced view of Greek culture, showing how human irrationali

In [14]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: If you read **"The Greeks and the Irrational" by E.R. Dodds**, you'll gain a deeper and more nuanced understanding of ancient Greek history and culture, particularly the interplay between rationality and irrationality in their worldview. Here’s what you’ll learn:

### 1. **The Role of Emotions in Greek Society**  
   - You'll explore how emotions such as shame, guilt, and anger influenced Greek behavior and social norms.
   - Dodds dives into the psychological and moral frameworks of the Greeks, showing how these emotions shaped their decisions and values.

### 2. **Religion and Supernatural Beliefs**  
   - The book examines the importance of religious practices, divine possession, and the belief in oracles in Greek society.
   - You'll learn how rituals and supernatural elements were woven into their daily lives, politics, and even philosophy.

### 3. **The Influence of Non-Rational Forces on Literatur

In [15]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Certainly! If you're interested in exploring more books about the interplay of rationality, irrationality, philosophy, and religion in Greek history and culture, here are some excellent options:

---

### **Philosophy and Thought in Ancient Greece**  
1. **"Greek Religion" by Walter Burkert**  
   - A comprehensive exploration of ancient Greek religion, rituals, myths, and practices. Burkert examines how religion influenced Greek society and thought, offering detailed insights into their beliefs about gods, fate, and the supernatural.

2. **"The Birth of Tragedy" by Friedrich Nietzsche**  
   - Nietzsche examines the tension between the rational, ordered "Apollonian" aspects of Greek culture and the chaotic, emotional "Dionysian" elements, particularly in Greek tragedy. A philosophical classic that delves deeply into Greek art and thought.

3. **"The Sophists" by W.K.C. Guthrie**  
   - A detailed account of t

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!


In [16]:
print(chat_history)

<chat_history><message role="system"><text>You are a helpful chatbot who is good about giving book recommendations.</text></message><message role="assistant"><text>Of course! I'd love to help. Could you tell me a bit more about what you're looking for? Are you into fiction or nonfiction? Any specific genres, themes, or authors you enjoy?</text></message><message role="user"><text>I love history and philosophy, I'd like to learn something new about Greece, any suggestion?</text></message><message role="assistant"><text>Absolutely! Here are some excellent book recommendations that delve into Greek history and philosophy:

### Greek History:
1. **"The Histories" by Herodotus**  
   Known as the "Father of History," Herodotus provides fascinating insights into ancient Greece and its neighboring civilizations, blending historical accounts with cultural observations.

2. **"The Peloponnesian War" by Thucydides**  
   A detailed and analytical account of the conflict between Athens and Sparta